## Transformée de Hough naive en c++ #kernel c++ 17 xeus cling sous Fedora

## Prototype du c++ , à faire un fichier propre sur vscode ou au bloc note après 
## Lucas ou Frobenius faites ca le matin si possible sv je serais surement en train de dormir

In [1]:
#pragma cling add_include_path("/usr/include/eigen3")
#pragma cling add_include_path("/usr/include/opencv4")
#pragma cling load("opencv_core")
#pragma cling load("opencv_imgproc")
#pragma cling load("opencv_highgui")


In [2]:
#include <Eigen/Dense>
#include <iostream>
#include <vector>
#include <cstdlib>
#include <ctime>
#include <cmath>
#include <opencv2/opencv.hpp>

### Test matrice aleatoire en 100*100

In [3]:
int lignes = 100;
int colonnes = 100;

In [4]:
std::vector<std::vector<int>> matrice(lignes, std::vector<int>(colonnes));

    // init du nmbre aler
    std::srand(std::time(0));

    // ca remplis
    for (int i = 0; i < lignes; ++i) {
        for (int j = 0; j < colonnes; ++j) {
            matrice[i][j] = std::rand() % 2;  // 0 ou 1 
        }
    }

In [5]:
for (int i = 0; i < lignes; ++i) {
    for (int j = 0; j < colonnes; ++j) {
        std::cout << matrice[i][j] << " ";
    }
    std::cout << std::endl;
}


0 0 1 0 0 1 0 1 1 0 0 0 0 1 1 0 1 1 1 0 0 1 0 1 1 0 0 0 1 0 0 0 1 1 0 1 1 0 0 0 0 1 0 1 0 0 1 1 1 0 0 1 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 1 0 0 0 1 0 1 1 0 0 0 1 0 0 0 1 1 1 0 0 0 1 0 1 0 1 0 1 0 1 1 1 
1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 0 0 0 1 1 0 1 0 0 1 0 1 1 0 1 0 1 1 0 1 1 1 0 1 0 0 1 0 0 0 1 0 1 1 1 0 1 1 0 0 0 0 1 0 1 0 
0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 1 1 1 1 1 0 0 0 0 1 0 0 1 1 0 1 0 0 1 0 0 1 1 0 1 0 1 0 1 1 1 0 1 0 1 0 0 1 1 0 0 1 0 0 1 1 1 1 1 0 1 0 1 0 0 0 0 0 1 1 1 0 1 0 0 1 1 1 0 0 1 1 1 0 1 0 1 0 1 0 
0 0 0 1 0 1 1 1 1 0 0 0 0 0 1 1 1 0 0 1 0 1 0 1 1 1 0 0 1 1 1 1 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 1 1 0 0 1 1 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 1 1 1 1 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 
0 0 1 0 0 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 1 1 0 0 1 0 1 0 0 0 1 0 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 0 0 1 0 0 0 0 1 0 1 1 0 0 0 1 0 1 0 0 0 1 0 1 0 0 0 1 1 0 0 0 0 1 0 0 1 

In [10]:
void transformerHough(const std::vector<std::vector<int>>& matrice,
                      int lignes,
                      int colonnes,
                      int rhoMax,
                      const std::vector<double>& theta,
                      Eigen::MatrixXi& accumulateur) {
    accumulateur = Eigen::MatrixXi::Zero(2 * rhoMax, theta.size());

    std::vector<int> indicesX;
    std::vector<int> indicesY;

    // ca pushback
    for (int i = 0; i < lignes; ++i) {
        for (int j = 0; j < colonnes; ++j) {
            if (matrice[i][j] != 0) {
                indicesX.push_back(j);  
                indicesY.push_back(i);  
            }
        }
    }

    // Remplissage de l'accumulateur
    for (size_t i = 0; i < indicesX.size(); ++i) {
        int x = indicesX[i];
        int y = indicesY[i];

        for (size_t j = 0; j < theta.size(); ++j) {
            double rho = x * std::cos(theta[j]) + y * std::sin(theta[j]);
            int rhoIndex = static_cast<int>(std::round(rho)) + rhoMax;

            if (rhoIndex >= 0 && rhoIndex < 2 * rhoMax) {
                accumulateur(rhoIndex, j) += 1;
            }
        }
    }

    // Affiche la matrice entière
    std::cout << "Matrice " << accumulateur.rows()
              << " x " << accumulateur.cols() << ") :\n";
    std::cout << accumulateur << std::endl;

    std::cout << "max : "
              << accumulateur.maxCoeff() << std::endl;
}


In [11]:
int main() {
    const int lignes = 100;
    const int colonnes = 100;

    std::vector<std::vector<int>> matrice(lignes, std::vector<int>(colonnes));

    std::srand(std::time(0));  // init aléatoire

    // ca remplis
    for (int i = 0; i < lignes; ++i) {
        for (int j = 0; j < colonnes; ++j) {
            matrice[i][j] = std::rand() % 2;
        }
    }

    // 0 180deg
    std::vector<double> theta;
    for (int i = 0; i < 100; ++i) {
        theta.push_back(i * M_PI / 180.0);
    }

    int rhoMax = std::ceil(std::sqrt(lignes * lignes + colonnes * colonnes));

    Eigen::MatrixXi accumulateur;

    transformerHough(matrice, lignes, colonnes, rhoMax, theta, accumulateur);

    return 0;
}

In [12]:
main()

Matrice 284 x 100) :
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0

0

### Reste à faire ; lire l'image ppm en c++